# Домашнее задание: натренировать модель!

Ваше задание натренировать модель генерировать гороскопы.

* __dataset:__ датасет гороскопов https://huggingface.co/datasets/dkagramanyan/horoscopes_ru
dataset = load_dataset("dkagramanyan/horoscopes_ru")
* __short lines:__ берите только первые 512 символов
* __adapter type:__ используйте LoRA как выше __и хотя бы один из:__
   - экстра adapter на lm_head
   - экстра adapter на MLP components (mlp.*)
   - LoRA из peft по аналогии  

PEFT:
1. Обучите модель (например, https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora ) с помощью LoRA аналогично коду из ноутбука на датасете гороскопов, оберните слои attention.

2. Обучите модель с помощью peft.LoraConfig.

3. Сравните результаты на нескольких промптах.

Решение в формате ссылки на Google Colaboratory необходимо выложить в соответствующее задание на учебном портале.

Дедлайн сдачи: 29 февраля 23:59

### ClearML

In [ ]:
!pip install clearml

In [ ]:
class InitClearml:
    def __init__(self, project_name: str, task_name: str):
        self.project_name = project_name
        self.task_name  = task_name

    def init_task(self):
        self.clearml_available = False
        try:
            from clearml import Task

            web_server = "https://app.clear.ml"  # @param {type:"string"}
            api_server = "https://api.clear.ml"  # @param {type:"string"}
            files_server = "https://files.clear.ml"  # @param {type:"string"}
            access_key = ""  # @param {type:"string"}
            secret_key = ""  # @param {type:"string"}

            if access_key == "" or secret_key == "":
                raise Exception("Empty access_key or secret_key")

            Task.set_credentials(
                web_host=web_server,
                api_host=api_server,
                files_host=files_server,
                key=access_key,
                secret=secret_key,
            )

            self.clearml_available = True
            return Task

        except ImportError:
            return
        except Exception as e:
            print(e)

    def init_experiment(self, Task):
        if self.clearml_available:
            task = Task.init(project_name=self.project_name, task_name=self.task_name)
            return task

### Скачивание необходимых библиотек

In [ ]:
!pip install datasets peft transformers optimum bitsandbytes==0.41.2.post2 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.6 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.1 MB/s eta 0:00:00


# Импорт библиотек

In [ ]:
from math import exp
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
import bitsandbytes as bnb
from transformers.trainer_pt_utils import get_parameter_names
from peft import LoraConfig, TaskType
from peft import get_peft_model

# Основная часть

In [ ]:
def tokenize(tokenizer, element, prompt="Гороскоп на завтра: "):
    if isinstance(element["text"], list):
        text = [prompt + s for s in element["text"]]
    else:
        text = prompt + element["text"]

    outputs = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors='pt',
        add_special_tokens=True
    )
    return outputs

def generate(model, prefix="Гороскоп на завтра: ", text="", **kwargs):
    model.eval()
    inputs = tokenizer(
        prefix + text,
        return_tensors='pt',
        add_special_tokens=True
    )

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=100,
            eos_token_id=50257,
            **kwargs
        )

        return tokenizer.batch_decode(
            outputs_ids.detach().cpu().numpy(),
            skip_special_tokens=True
        )

## Данные

In [ ]:
data = load_dataset("dkagramanyan/horoscopes_ru")
data

DatasetDict({
    train: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 66501
    })
    test: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 6976
    })
})

## Модель и токенайзер

In [ ]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Токенизация данных

In [ ]:
train_ds = data["train"].shuffle(seed=1).select(range(400)).map(
    lambda x: tokenize(tokenizer=tokenizer, element=x), batched=True, remove_columns=data["train"].column_names
)
test_ds = data["test"].shuffle(seed=1).select(range(100)).map(
    lambda x: tokenize(tokenizer=tokenizer, element=x), batched=True, remove_columns=data["test"].column_names
)

## Часть 1
> Обучите модель с помощью LoRA аналогично коду из ноутбука на датасете гороскопов, оберните слои attention.

In [ ]:
clearml_logging = InitClearml("FinTech VTB", "IlyaGusev/saiga2_7b_lora(custom_class_LoRA)_2")
Task = clearml_logging.init_task()
task = clearml_logging.init_experiment(Task)
clearml_available = clearml_logging.clearml_available

In [ ]:
logger = task.get_logger()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    offload_state_dict=True,
    low_cpu_mem_usage=True,
    load_in_4bit=True,
    torch_dtype=torch.float32
)

for param in model.parameters():
    param.requires_grad = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


2024-02-29 10:59:08,778 - clearml.model - INFO - Selected model id: 24e198bb75434e0bb7470248c00eb8cc
2024-02-29 10:59:26,289 - clearml.model - INFO - Selected model id: e1b30cc602d94777b2e9db8dd60fe524
2024-02-29 10:59:35,940 - clearml.model - INFO - Selected model id: 755908d6613942cb9a3e083e5472eb01


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, i

In [ ]:
test_init_model = generate(model, prefix="Гороскоп на завтра: ")

test_init_model

['Гороскоп на завтра: 2022-01-16\n Unterscheidung: 100%\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с ю']

In [ ]:
if clearml_available:
    logger.report_text(test_init_model)

['Гороскоп на завтра: 2022-01-16\n Unterscheidung: 100%\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с юга.\n\nВысокая температура, солнечный день, ветер с ю']


### LoRA

#### Custom class LoRA

In [ ]:
class LoRALayer(nn.Module):
    def __init__(self, module, rank: int, alpha: float):
        super().__init__()
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))
        self.alpha = alpha

    def forward(self, x):
        adapter_output = torch.matmul(x, self.adapter_A)
        adapter_output = torch.matmul(adapter_output, self.adapter_B)
        x = self.alpha * adapter_output

        return x


class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank: int, alpha: float = 1):
        super().__init__()
        self.module = linear
        self.lora = LoRALayer(
            self.module, rank, alpha
        )

    def forward(self, x):
        return self.module(x) + self.lora(x)

#### Тест LoRA

In [ ]:
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LinearWithLoRA(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1)

test_adapter.lora.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.lora.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.lora.adapter_A, test_adapter.lora.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.lora.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.lora.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


#### Подставляем LoRA в оригинальную нейросеть

In [ ]:
from functools import partial
task_params = {}
lora_params = {}

# logger
lora_params["rank"] = 16
lora_params["alpha"] = 1
lora_params["dropout"] = None
lora_params["query"] = True
lora_params["key"] = True
lora_params["value"] = True
lora_params["projection"] = True
lora_params["mlp"] = False
lora_params["head"] = False

layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_params["rank"], alpha=lora_params["alpha"])

for layer in model.model.layers:
    if lora_params["query"]:
        layer.self_attn.q_proj = assign_lora(layer.self_attn.q_proj)
    if lora_params["key"]:
        layer.self_attn.k_proj = assign_lora(layer.self_attn.k_proj)
    if lora_params["value"]:
        layer.self_attn.v_proj = assign_lora(layer.self_attn.v_proj)
    if lora_params["projection"]:
        layer.self_attn.o_proj = assign_lora(layer.self_attn.o_proj)
    if lora_params["mlp"]:
        layer.mlp.gate_proj = assign_lora(layer.mlp.gate_proj)
        layer.mlp.up_proj = assign_lora(layer.mlp.up_proj)
        layer.mlp.down_proj = assign_lora(layer.mlp.down_proj)
if lora_params["head"]:
    model.lm_head = assign_lora(model.lm_head)

task_params["LoRA settings"] = lora_params
# if clearml_available:
#     task.connect(task_params)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LinearWithLoRA(
            (module): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=4096, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (lora): LoRALayer()
          )
          (k_proj): LinearWithLoRA(
            (module): lora.Linear4bit(
              (base_la

In [ ]:
batch = tokenizer("Это будет что-то секретное:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Градиенты считаются")

Градиенты считаются


### Подготовка к обучению

Коллатор

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Параметры обучения

In [ ]:
# Определение параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    logging_dir="./runs",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    save_steps=20,
    logging_steps=2,
    learning_rate=5e-3,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    save_total_limit=4,
    prediction_loss_only=True,
    fp16=True,
    save_safetensors=False
)

Оптимайзер Adam 8 bit

In [ ]:
decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate

adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

Посмотрим, сколько обучаемых параметров в NN. Есть вероятность, что все параметры не очень правильно посчитаны. (Мб надо надо умножить на 2 до того, как добавил свой слой LoRA)

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
params = sum(p.numel() for p in model.parameters())
print(f"trainable params: {trainable_params:_}\t all params: {params:_}\t trainable%: {trainable_params/params}")

trainable params: 16_777_216	 all params: 3_533_967_360	 trainable%: 0.004747416795609567


### Trainer

In [ ]:
model.config.use_cache = False
# model._hf_peft_config_loaded = True
trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    optimizers=(adam_bnb_optim, None)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

2024-02-29 10:59:51,564 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Step,Training Loss
2,2.278200
4,2.161800
6,2.166700
8,2.087900
10,2.010900
12,2.074100
14,1.968900
16,1.959200
18,1.886900
20,1.942300


2024-02-29 11:00:53,995 - clearml - INFO - inf value encountered. Reporting it as '0.0'. Use clearml.Logger.set_reporting_inf_value to assign another value


/opt/conda/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


TrainOutput(global_step=40, training_loss=1.951166146993637, metrics={'train_runtime': 628.5159, 'train_samples_per_second': 0.636, 'train_steps_per_second': 0.064, 'total_flos': 8160335521382400.0, 'train_loss': 1.951166146993637, 'epoch': 1.0})

### Немного размышлений

Слои обучаются те, которые как-раз такие были вставлены

In [ ]:
trainable_layers = []

for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_layers.append(name)

print("Trainable Layers:")
for layer in trainable_layers:
    print(layer)

Trainable Layers:
model.layers.0.self_attn.q_proj.lora.adapter_A
model.layers.0.self_attn.q_proj.lora.adapter_B
model.layers.0.self_attn.k_proj.lora.adapter_A
model.layers.0.self_attn.k_proj.lora.adapter_B
model.layers.0.self_attn.v_proj.lora.adapter_A
model.layers.0.self_attn.v_proj.lora.adapter_B
model.layers.0.self_attn.o_proj.lora.adapter_A
model.layers.0.self_attn.o_proj.lora.adapter_B
model.layers.1.self_attn.q_proj.lora.adapter_A
model.layers.1.self_attn.q_proj.lora.adapter_B
model.layers.1.self_attn.k_proj.lora.adapter_A
model.layers.1.self_attn.k_proj.lora.adapter_B
model.layers.1.self_attn.v_proj.lora.adapter_A
model.layers.1.self_attn.v_proj.lora.adapter_B
model.layers.1.self_attn.o_proj.lora.adapter_A
model.layers.1.self_attn.o_proj.lora.adapter_B
model.layers.2.self_attn.q_proj.lora.adapter_A
model.layers.2.self_attn.q_proj.lora.adapter_B
model.layers.2.self_attn.k_proj.lora.adapter_A
model.layers.2.self_attn.k_proj.lora.adapter_B
model.layers.2.self_attn.v_proj.lora.adapt

Т.к. в чекпоинтах сохраняется названия других слоев, то можно их поменять их название, допуская то, что все-таки наши веса сохраняются.  
Проделываем все эти манипуляции и пытаемся снова перезапустить ноутбук и проинициализировать модель.  
В итоге получается, что *Trainer* сохранял веса от LoRA, которая уже до этого была заложена в модели.  
Получается, что надо писать свой кастомный класс, который будет сохранять эти веса при обучении, чтобы потом их можно было проинициализировать.

In [ ]:
from itertools import product

In [ ]:
test_head = ["q_proj", "k_proj", "v_proj", "o_proj"]
lora_map = ["A", "B"]
layres_test = list(range(32))

state_dict = torch.load("./results/checkpoint-100/adapter_model.bin")


In [ ]:
all_combinations = list(product(test_head, lora_map, layres_test))

In [ ]:
all_combinations[0], all_combinations[31]

(('q_proj', 'A', 0), ('q_proj', 'A', 31))

In [ ]:
len(all_combinations), len(state_dict)

(256, 256)

In [ ]:
state_dict["base_model.model.model.layers.0.self_attn.q_proj.module.lora_A.weight"].size()

torch.Size([16, 4096])

In [ ]:
model.model.layers[0].self_attn.q_proj.lora.adapter_A.size()

torch.Size([4096, 8])

In [ ]:
param_mapping = {
    f"base_model.model.model.layers.{l}.self_attn.{h}.module.lora_{a}.weight": f"model.layers.{l}.self_attn.{h}.lora.adapter_{a}"
    for h, a, l in all_combinations
}

# model.layers.0.self_attn.q_proj.lora.adapter_B

In [ ]:
len(param_mapping)

256

In [ ]:
mapped_state_dict = {}
for src_param_name, dst_param_name in param_mapping.items():
    if src_param_name in state_dict:
        mapped_state_dict[dst_param_name] = state_dict[src_param_name]

In [ ]:
torch.save(mapped_state_dict, "./results/checkpoint-100/adapter_model_fix.bin")

In [ ]:
model.load_state_dict(torch.load("./results/checkpoint-100/adapter_model_fix.bin"), strict=False)

RuntimeError: Error(s) in loading state_dict for LlamaForCausalLM:
	size mismatch for model.layers.0.self_attn.q_proj.lora.adapter_A: copying a param with shape torch.Size([16, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 8]).
	size mismatch for model.layers.0.self_attn.q_proj.lora.adapter_B: copying a param with shape torch.Size([4096, 16]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for model.layers.0.self_attn.k_proj.lora.adapter_A: copying a param with shape torch.Size([16, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 8]).
	size mismatch for model.layers.0.self_attn.k_proj.lora.adapter_B: copying a param with shape torch.Size([4096, 16]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for model.layers.0.self_attn.v_proj.lora.adapter_A: copying a param with shape torch.Size([16, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 8])

In [ ]:
model.model.layers[0].self_attn.q_proj.lora.adapter_B

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)

### Eval Test (Perplexity)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.14


### Некоторые промпты для этой модели

In [ ]:
test_1_finetune_model_1 = generate(model)

test_1_finetune_model_1

['Гороскоп на завтра:  Сегодня вы можете ощутить недостаток внимания со стороны окружающих. В первую очередь это касается вашей семьи, друзей, партнеров по делу. Не исключены и недоразумения в отношениях с коллегами. Не стоит забывать, что ваша работа может быть не только важной, но и опасной. Не']

In [ ]:
test_2_finetune_model_1 = generate(model, text="Близнецам лучше провести день", prefix="")

test_2_finetune_model_1

['Близнецам лучше провести день в домашних условиях, вдали от любопытства и внимания окружающих. Возможны споры и разногласия, но в целом это хороший день для семейных мелких поездок, покупки подарков, поиска новых интересных книг, покупки домашних приборов, покупки продуктов для домашнего приготовления. В этот']

In [ ]:
test_3_finetune_model_1 = generate(
    model,
    text="Козерогам в этот день",
    do_sample=True,
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
)

test_3_finetune_model_1

['Гороскоп на завтра: Козерогам в этот день не стоит забывать о том, что любовь и сексуальные отношения могут привести к серьезным последствиям. Возможны разногласия с партнером по чувствам из-за различий во взглядах на семейную жизнь или воспитание детей. Не рекомендуется ухаживать за домашними животными - это может вызвать']

In [ ]:
test_4_finetune_model_1 = generate(
    model,
    text="Весы",
    do_sample=True,
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
)

test_4_finetune_model_1

['Гороскоп на завтра: Весы сегодняшнего дня могут столкнуться с неожиданными препятствиями в своих планах. Возможно, вам придется пересмотреть свою стратегию или выбрать другой путь для достижения поставленной цели. Не исключен конфликт со своим партнером (в любом отношении). Постарайтесь избежа']

In [ ]:
if clearml_available:
    logger.report_text(test_1_finetune_model_1)
    logger.report_text(test_2_finetune_model_1)
    logger.report_text(test_3_finetune_model_1)
    logger.report_text(test_4_finetune_model_1)

['Гороскоп на завтра:  Сегодня вы можете ощутить недостаток внимания со стороны окружающих. В первую очередь это касается вашей семьи, друзей, партнеров по делу. Не исключены и недоразумения в отношениях с коллегами. Не стоит забывать, что ваша работа может быть не только важной, но и опасной. Не']
['Близнецам лучше провести день в домашних условиях, вдали от любопытства и внимания окружающих. Возможны споры и разногласия, но в целом это хороший день для семейных мелких поездок, покупки подарков, поиска новых интересных книг, покупки домашних приборов, покупки продуктов для домашнего приготовления. В этот']
['Гороскоп на завтра: Козерогам в этот день не стоит забывать о том, что любовь и сексуальные отношения могут привести к серьезным последствиям. Возможны разногласия с партнером по чувствам из-за различий во взглядах на семейную жизнь или воспитание детей. Не рекомендуется ухаживать за домашними животными - это может вызвать']
['Гороскоп на завтра: Весы сегодняшнего дня могут столкн

In [ ]:
del model, trainer, training_args
gc.collect()
torch.cuda.empty_cache()

## Часть 2
> Обучите модель с помощью peft.LoraConfig.

In [ ]:
clearml_logging = InitClearml("FinTech VTB", "IlyaGusev/saiga2_7b_lora(default_class_LoRA)")
Task = clearml_logging.init_task()
task = clearml_logging.init_experiment(Task)
clearml_available = clearml_logging.clearml_available

ClearML Task: created new task id=8734cccdb71b4809aa60df161cfc423f
2024-02-29 12:25:48,810 - clearml.Repository Detection - WARNING - Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf
2024-02-29 12:25:48,815 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/a0e39586066d49ac87ba40b5f2c1ec16/experiments/8734cccdb71b4809aa60df161cfc423f/output/log


In [ ]:
logger = task.get_logger()

### Конфиг LoRA

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

### Модель

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    offload_state_dict=True,
    low_cpu_mem_usage=True,
    load_in_4bit=True,
    torch_dtype=torch.float32
)

# for param in model.parameters():
#     param.requires_grad = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model = get_peft_model(model, peft_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


2024-02-29 12:26:21,922 - clearml.model - INFO - Selected model id: 24e198bb75434e0bb7470248c00eb8cc
2024-02-29 12:26:41,638 - clearml.model - INFO - Selected model id: e1b30cc602d94777b2e9db8dd60fe524
2024-02-29 12:26:51,660 - clearml.model - INFO - Selected model id: 755908d6613942cb9a3e083e5472eb01


### Подготовка к обучению

Коллатор

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Параметры обучения

In [ ]:
# Определение параметров обучения
training_args = TrainingArguments(
    output_dir='./results_defLoRA',
    logging_dir="./runs",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    save_steps=20,
    logging_steps=2,
    learning_rate=5e-3,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    save_total_limit=4,
    prediction_loss_only=True,
    fp16=True,
    save_safetensors=False
)

Adam 8 bit

In [ ]:
decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate

adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

Посмотрим, сколько обучающихся параметров

In [ ]:
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.24836028248556738


In [ ]:
model.config.use_cache = False
trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    optimizers=(adam_bnb_optim, None)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

2024-02-29 12:27:04,760 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


/opt/conda/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
2,2.325700
4,2.307800
6,2.282200
8,2.250500
10,2.266100
12,2.319500
14,2.253300
16,2.328300
18,2.259700
20,2.255500


TrainOutput(global_step=40, training_loss=2.175976014137268, metrics={'train_runtime': 658.9761, 'train_samples_per_second': 0.607, 'train_steps_per_second': 0.061, 'total_flos': 8139719678361600.0, 'train_loss': 2.175976014137268, 'epoch': 1.0})

### Eval Test (Perplexity)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {exp(eval_results['eval_loss']):.2f}")

Perplexity: 7.41


### Некоторые промты для этой модели

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
test_1_finetune_model_2 = generate(model)

test_1_finetune_model_2

['Гороскоп на завтра: ♈ Январский солнечный день, который начинается с утреннего пробуждения, может быть очень активным и насыщенным. Hinweis: В этот день вы можете почувствовать себя особенно энергичным и уверенным в себе. Этот день может быть очень продуктивным для вас, если вы сможете использовать энергию, которую вы']

In [ ]:
test_2_finetune_model_2 = generate(model, text="Близнецам лучше провести день", prefix="")

test_2_finetune_model_2

['Близнецам лучше провести день в доме, чем на улице. Hinweis: в детской игрушке нет ничего небезопасного.\nВыбор игрушек для девочек: куклы, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, иг']

In [ ]:
test_3_finetune_model_2 = generate(
    model,
    text="Козерогам в этот день",
    do_sample=True,
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
)

test_3_finetune_model_2

['Гороскоп на завтра: Козерогам в этот день предстоит тяжёлое испытание, которое может привести к серьёзным последствиям для их карьеры и финансовой стабильности. февральские солнцепоклонники могут столкнуться с необходимостью выбирать между двумя противоречащими друг другу вариантами развития своих профессиональных или личных интересов']

In [ ]:
test_4_finetune_model_2 = generate(
    model,
    text="Весы",
    do_sample=True,
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
)

test_4_finetune_model_2

['Гороскоп на завтра: Весы вашего успеха сегодня весьма благоприятны, поэтому вы имеете все основания рассчитывать на хорошие результаты в любых сферах деятельности. prüfen Sie Ihren Horoskop jetzt kostenlos auf horoscope24.de!\nПоследние 10 дней были очень напряженными и требовательными по отношению к Вашей энер']

In [ ]:
if clearml_available:
    logger.report_text(test_1_finetune_model_2)
    logger.report_text(test_2_finetune_model_2)
    logger.report_text(test_3_finetune_model_2)
    logger.report_text(test_4_finetune_model_2)
task.close()

['Гороскоп на завтра: ♈ Январский солнечный день, который начинается с утреннего пробуждения, может быть очень активным и насыщенным. Hinweis: В этот день вы можете почувствовать себя особенно энергичным и уверенным в себе. Этот день может быть очень продуктивным для вас, если вы сможете использовать энергию, которую вы']
['Близнецам лучше провести день в доме, чем на улице. Hinweis: в детской игрушке нет ничего небезопасного.\nВыбор игрушек для девочек: куклы, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, иг']
['Гороскоп на завтра: Козерогам в этот день предстоит тяжёлое испытание, которое может привести к серьёзным последствиям для их карьеры и финансовой стабильности. февральские солнцепоклонники могут столкнуться с необходимостью выбирать между двумя противоречащими друг другу вариантами развития своих профессиональных или личных интересов']
['Гороскоп на завтра: Весы вашего успеха сегодня весьма благопри

# Выводы

In [3]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">FINETUNING CUSTOM LoRA</th>
    <th style="text-align: center; border:1px solid black">FINETUNING DEFAULT LoRA</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

asnwers_custom_lora = [
    "Гороскоп на завтра:  Сегодня вы можете ощутить недостаток внимания со стороны окружающих. \n В первую очередь это касается вашей семьи, друзей, партнеров по делу. \n Не исключены и недоразумения в отношениях с коллегами. \n Не стоит забывать, что ваша работа может быть не только важной, но и опасной. Не",
    "Близнецам лучше провести день в домашних условиях, вдали от любопытства и внимания окружающих. \n Возможны споры и разногласия, но в целом это хороший день для семейных мелких поездок, покупки подарков, поиска новых интересных книг, \nпокупки домашних приборов, покупки продуктов для домашнего приготовления. В этот",
    "Гороскоп на завтра: Козерогам в этот день не стоит забывать о том, что любовь и сексуальные отношения могут привести к серьезным последствиям. \n Возможны разногласия с партнером по чувствам из-за различий во взглядах на семейную жизнь \n или воспитание детей. Не рекомендуется ухаживать за домашними животными - это может вызвать",
    "Гороскоп на завтра: Весы сегодняшнего дня могут столкнуться с неожиданными препятствиями в своих планах. \n Возможно, вам придется пересмотреть свою стратегию или выбрать другой путь для достижения поставленной цели. Не исключен конфликт со своим партнером (в любом отношении). Постарайтесь избежа"
]

asnwers_default_lora = [
    "Гороскоп на завтра: ♈ Январский солнечный день, который начинается с утреннего пробуждения, \nможет быть очень активным и насыщенным. Hinweis: В этот день вы можете почувствовать себя особенно энергичным и уверенным в себе. \n Этот день может быть очень продуктивным для вас, если вы сможете использовать энергию, которую вы",
    "Близнецам лучше провести день в доме, чем на улице. \n Hinweis: в детской игрушке нет ничего небезопасного.\nВыбор игрушек для девочек: куклы, игрушки для маленьких девочек, игрушки для маленьких девочек, \n игрушки для маленьких девочек, игрушки для маленьких девочек, иг",
    "Гороскоп на завтра: Козерогам в этот день предстоит тяжёлое испытание, \n которое может привести к серьёзным последствиям для их карьеры и финансовой стабильности. февральские солнцепоклонники могут \n столкнуться с необходимостью выбирать между двумя противоречащими друг другу вариантами развития \n своих профессиональных или личных интересов",
    "Гороскоп на завтра: Весы вашего успеха сегодня весьма благоприятны, поэтому вы имеете \n все основания рассчитывать на хорошие результаты в любых сферах деятельности. prüfen Sie Ihren Horoskop jetzt kostenlos auf horoscope24.de! \n Последние 10 дней были очень напряженными и требовательными по отношению к Вашей энер"
]

promts = ['Гороскоп на завтра:', 'Близнецам лучше провести день', 'Гороскоп на завтра: Козерогам в этот день', 'Гороскоп на завтра: Весы']

for prompt, asnwer_custom_lora, asnwer_default_lora in zip(promts, asnwers_custom_lora, asnwers_default_lora):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prompt, asnwer_custom_lora, asnwer_default_lora))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,FINETUNING CUSTOM LoRA,FINETUNING DEFAULT LoRA
`Гороскоп на завтра:`,"Гороскоп на завтра: Сегодня вы можете ощутить недостаток внимания со стороны окружающих. В первую очередь это касается вашей семьи, друзей, партнеров по делу. Не исключены и недоразумения в отношениях с коллегами. Не стоит забывать, что ваша работа может быть не только важной, но и опасной. Не","Гороскоп на завтра: ♈ Январский солнечный день, который начинается с утреннего пробуждения, может быть очень активным и насыщенным. Hinweis: В этот день вы можете почувствовать себя особенно энергичным и уверенным в себе. Этот день может быть очень продуктивным для вас, если вы сможете использовать энергию, которую вы"
`Близнецам лучше провести день`,"Близнецам лучше провести день в домашних условиях, вдали от любопытства и внимания окружающих. Возможны споры и разногласия, но в целом это хороший день для семейных мелких поездок, покупки подарков, поиска новых интересных книг, покупки домашних приборов, покупки продуктов для домашнего приготовления. В этот","Близнецам лучше провести день в доме, чем на улице. Hinweis: в детской игрушке нет ничего небезопасного. Выбор игрушек для девочек: куклы, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, игрушки для маленьких девочек, иг"
`Гороскоп на завтра: Козерогам в этот день`,"Гороскоп на завтра: Козерогам в этот день не стоит забывать о том, что любовь и сексуальные отношения могут привести к серьезным последствиям. Возможны разногласия с партнером по чувствам из-за различий во взглядах на семейную жизнь или воспитание детей. Не рекомендуется ухаживать за домашними животными - это может вызвать","Гороскоп на завтра: Козерогам в этот день предстоит тяжёлое испытание, которое может привести к серьёзным последствиям для их карьеры и финансовой стабильности. февральские солнцепоклонники могут столкнуться с необходимостью выбирать между двумя противоречащими друг другу вариантами развития своих профессиональных или личных интересов"
`Гороскоп на завтра: Весы`,"Гороскоп на завтра: Весы сегодняшнего дня могут столкнуться с неожиданными препятствиями в своих планах. Возможно, вам придется пересмотреть свою стратегию или выбрать другой путь для достижения поставленной цели. Не исключен конфликт со своим партнером (в любом отношении). Постарайтесь избежа","Гороскоп на завтра: Весы вашего успеха сегодня весьма благоприятны, поэтому вы имеете все основания рассчитывать на хорошие результаты в любых сферах деятельности. prüfen Sie Ihren Horoskop jetzt kostenlos auf horoscope24.de! Последние 10 дней были очень напряженными и требовательными по отношению к Вашей энер"


https://app.clear.ml/projects/a0e39586066d49ac87ba40b5f2c1ec16/experiments/17949fa0a99942738ae562a57b29013b/output/execution  
https://app.clear.ml/projects/a0e39586066d49ac87ba40b5f2c1ec16/experiments/8734cccdb71b4809aa60df161cfc423f/output/execution  

Для второй модели, наверное, можно было дать больше данных, чтобы ответы были более вариативные. Учится более плавно, чем первая модель по лоссу.  
Для первой модели с увеличением данных градиентам становится "плохо". Скорей всего, можно было добавить dropout.  
При прочих равных у первой ответы чуть лучше, но опять же predict также надо преднастраивать для каждой модели.  